In [17]:
import os
from square.client import Client
import config
from square.http.auth.o_auth_2 import BearerAuthCredentials

client = Client(
    bearer_auth_credentials=BearerAuthCredentials(
        access_token=config.square_access_token
    ),
    environment=config.square_environment,
)


ModuleNotFoundError: No module named 'config'